负责把vnpy的单个symbol的所有子合约的日线 csv, 合成为复权形式的主力连续合约

不同symbol的主力合约切换是不同的,需要考虑不同的切换逻辑

这里使用通用逻辑: 如果n天volume总和被其他合约超过, 就切换合约

In [1]:
import glob
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
%matplotlib inline

In [15]:
# 单个symbol, vnpy格式的日线数据,
day_bar_data_folder = r"E:\mgc_day"
# 生成这个symbol所有子合约的csv path list
day_bar_data_path_list = glob.glob(os.path.join(day_bar_data_folder, "*.csv"))

In [16]:
day_bar_data_path_list[0:5]

['E:\\mgc_day\\MGCG11.csv',
 'E:\\mgc_day\\MGCG12.csv',
 'E:\\mgc_day\\MGCG13.csv',
 'E:\\mgc_day\\MGCG14.csv',
 'E:\\mgc_day\\MGCG15.csv']

In [38]:
# 这个方法不用了.感觉麻烦死了.还是自动的来,感觉问题也不是特别大
"""
# 需要手动的去找每年的主力合约. 自动的方法试过, 不靠谱.手动累一点但是心里踏实.
# 首先把每一年,每个子合约的所有交易量总和, 进行排序, 看一下每一年哪几个合约是主力合约
# 首先建立一个记录数据的字典
volume_dict = {
    "year": [],
    "symbol": [],
    "data_path":[],
    "volume": []
}
for path in day_bar_data_path_list:
    #按照string格式来找到年份,symbol
    volume_dict["year"].append(int(path[-6:-4]))
    volume_dict["symbol"].append(os.path.basename(path)[:-4])
    volume_dict["data_path"].append(path)
    #计算volume总和需要读取数据
    frame = pd.read_csv(path)
    volume_sum = frame.volume.sum()
    volume_dict["volume"].append(volume_sum)

# 把字典转换为dataframe
volume_frame = pd.DataFrame(volume_dict)
"""

'\n# 需要手动的去找每年的主力合约. 自动的方法试过, 不靠谱.手动累一点但是心里踏实.\n# 首先把每一年,每个子合约的所有交易量总和, 进行排序, 看一下每一年哪几个合约是主力合约\n# 首先建立一个记录数据的字典\nvolume_dict = {\n    "year": [],\n    "symbol": [],\n    "data_path":[],\n    "volume": []\n}\nfor path in day_bar_data_path_list:\n    #按照string格式来找到年份,symbol\n    volume_dict["year"].append(int(path[-6:-4]))\n    volume_dict["symbol"].append(os.path.basename(path)[:-4])\n    volume_dict["data_path"].append(path)\n    #计算volume总和需要读取数据\n    frame = pd.read_csv(path)\n    volume_sum = frame.volume.sum()\n    volume_dict["volume"].append(volume_sum)\n\n# 把字典转换为dataframe\nvolume_frame = pd.DataFrame(volume_dict)\n'

In [139]:
# 先把所有的数据读进缓存
data_dict = {}

for path in day_bar_data_path_list:
    frame = pd.read_csv(path)
    frame.loc[:, "date"] = pd.to_datetime(frame['date'], format='%Y-%m-%d')
    frame = frame.sort_values("date")
    frame.loc[:, "1_days_volume_sum"] = frame.loc[:, "volume"].rolling(min_periods=1, window=1).sum()
    data_dict[path] = frame


In [148]:
# 先创造一个空的主力连续合约
continuous_frame = pd.DataFrame()
# 初始化第一个合约
init_symbol_path = "E:\mgc_day\MGCZ10.csv"
main_frame = data_dict[init_symbol_path]
current_date = main_frame.iloc[0,: ].loc["date"]

# 切换合约的累计差值, 初始化为0
accumulative_adjust = 0
switch_close_price = 0


# 已经被切换的合约就不要再切换回来了
exclusive_list = [init_symbol_path]

# 初始化一个循环结束的flag
loop_end = False

# 初始第一根bar先推给主力合约
new_row = main_frame[main_frame["date"] == current_date].copy(deep=True)
# 要把累加值记录下来
new_row.loc[:, "accumulative_adjust"] = accumulative_adjust
continuous_frame = pd.concat([continuous_frame, new_row])

# switch = True

while(not loop_end):

    # 循环要从大于current date的数据开始, 不包括current date
    filter_main_frame = main_frame[main_frame["date"] > current_date]

    # 每次切换合约的累计差值计算, 第一次不算.
    if switch_close_price:
        accumulative_adjust += filter_main_frame.iloc[0,:].loc["open"] - switch_close_price
    row_count = 0
    for ix, row in filter_main_frame.iterrows():
        row_count += 1
        # 更新当前日子
        current_date = row.loc["date"].to_datetime64()
        # 先不管这么不多, 把当前行的数据推送到主力连续合约
        new_row = row.copy(deep=True)
        # 主力连续合约的价格需要根据复权的累计值调整
        for column in ['open', 'high', 'low', 'close']:
            new_row.loc[column] = row.loc[column] - accumulative_adjust
        # 要把累加值记录下来
        new_row.loc["accumulative_adjust"] = accumulative_adjust
        continuous_frame = continuous_frame.append(new_row)
        # 更新最后一根k线的close的值, 要原始的价格而不是调整后的
        switch_close_price = row.loc["close"]

        # 接着遍历所有合约的Frame去找有没有当天比当前主力1天volume总和大的合约
        # 如果找到了就隔一天换合约 
        current_volume_sum = row.loc["1_days_volume_sum"]
        # 创建一个记录所有超越当前1天volume的symbol dictionary
        switch_symbol_dict = {}
        for path in data_dict.keys():
            # 已经被切换的合约就不用再管了
            if path not in exclusive_list:
                frame = data_dict[path]
                if current_date in frame["date"].unique():
                    volume_sum = frame[frame["date"] == current_date].loc[:, "1_days_volume_sum"].iloc[0]
                    # volume_sum不能为空值
                    if volume_sum:
                        if current_volume_sum:
                            if volume_sum > current_volume_sum * 0.8:
                                # 把这个可能的合约加到dict里面去先
                                switch_symbol_dict[path] = volume_sum
        # 如果至少找到了一个可切换合约
        if switch_symbol_dict:
            # 按照1天交易量排序, 字典里volume sum最大的那个合约就切换那个
            sorted_symbol_list = sorted(switch_symbol_dict.items(), key=lambda x: x[1], reverse = True)
            new_main_path = sorted_symbol_list[0][0]
            # 切换合约
            exclusive_list.append(new_main_path)
            main_frame = data_dict[new_main_path]
            print(current_date)
            print(new_main_path)
            break
    if row_count == len(filter_main_frame):
        loop_end = True

2010-11-26T00:00:00.000000000
E:\mgc_day\MGCG11.csv
2011-01-26T00:00:00.000000000
E:\mgc_day\MGCJ11.csv
2011-03-27T00:00:00.000000000
E:\mgc_day\MGCM11.csv
2011-05-26T00:00:00.000000000
E:\mgc_day\MGCQ11.csv
2011-07-27T00:00:00.000000000
E:\mgc_day\MGCZ11.csv
2011-11-28T00:00:00.000000000
E:\mgc_day\MGCG12.csv
2012-01-27T00:00:00.000000000
E:\mgc_day\MGCJ12.csv
2012-03-28T00:00:00.000000000
E:\mgc_day\MGCM12.csv
2012-05-29T00:00:00.000000000
E:\mgc_day\MGCQ12.csv
2012-07-29T00:00:00.000000000
E:\mgc_day\MGCV12.csv
2012-07-30T00:00:00.000000000
E:\mgc_day\MGCZ12.csv
2012-11-28T00:00:00.000000000
E:\mgc_day\MGCG13.csv
2013-01-29T00:00:00.000000000
E:\mgc_day\MGCJ13.csv
2013-03-26T00:00:00.000000000
E:\mgc_day\MGCM13.csv
2013-05-29T00:00:00.000000000
E:\mgc_day\MGCQ13.csv
2013-07-28T00:00:00.000000000
E:\mgc_day\MGCZ13.csv
2013-11-26T00:00:00.000000000
E:\mgc_day\MGCG14.csv
2014-01-29T00:00:00.000000000
E:\mgc_day\MGCJ14.csv
2014-03-27T00:00:00.000000000
E:\mgc_day\MGCM14.csv
2014-05-25T0

In [149]:
continuous_frame.head()

,open,high,low,close,volume,date,exchange,open_interest,interval,symbol,1_days_volume_sum,accumulative_adjust
0,1320.0,1320.0,1318.1,1318.1,2.0,2010-10-03,Exchange.NYMEX,0.0,Interval.MINUTE,MGCZ10,2.0,0.0
1,1317.0,1317.1,1314.9,1316.7,19.0,2010-10-04,Exchange.NYMEX,0.0,Interval.MINUTE,MGCZ10,19.0,0.0
2,1329.1,1342.8,1328.7,1342.8,122.0,2010-10-05,Exchange.NYMEX,0.0,Interval.MINUTE,MGCZ10,122.0,0.0
3,1345.5,1350.0,1343.4,1345.7,220.0,2010-10-06,Exchange.NYMEX,0.0,Interval.MINUTE,MGCZ10,220.0,0.0
4,1361.1,1361.6,1327.7,1333.9,369.0,2010-10-07,Exchange.NYMEX,0.0,Interval.MINUTE,MGCZ10,369.0,0.0


In [150]:
continuous_frame.tail()

,open,high,low,close,volume,date,exchange,open_interest,interval,symbol,1_days_volume_sum,accumulative_adjust
276,1546.2,1549.4,1529.0,1540.9,53809.0,2020-05-11,Exchange.NYMEX,0.0,Interval.MINUTE,MGCM20,53809.0,163.2
277,1540.8,1553.3,1537.3,1542.6,48201.0,2020-05-12,Exchange.NYMEX,0.0,Interval.MINUTE,MGCM20,48201.0,163.2
278,1542.5,1565.0,1538.3,1558.9,67209.0,2020-05-13,Exchange.NYMEX,0.0,Interval.MINUTE,MGCM20,67209.0,163.2
279,1558.9,1583.3,1552.5,1579.2,64228.0,2020-05-14,Exchange.NYMEX,0.0,Interval.MINUTE,MGCM20,64228.0,163.2
280,1579.2,1582.2,1576.8,1582.0,3372.0,2020-05-15,Exchange.NYMEX,0.0,Interval.MINUTE,MGCM20,3372.0,163.2


In [151]:
continuous_frame.to_csv(r"E:\mgc_day_continuous\MGC.csv", index=False)